In [0]:
# ================= CELL 0: 环境清理脚本 (终极重置) =================
import os
import shutil

# 定义配置 (必须和 CELL 1 & 2 保持一致)
COURSE_NAME = "Data_Course_101"
CATALOG_NAME = "workspace"
SCHEMA_NAME = "default" # Volume 所在的 Schema
VOLUME_NAME = "course_data"
TARGET_CATALOG = "workspace"
TARGET_SCHEMA = "course_db"

# 动态获取当前用户身份
current_user = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()

user_workspace_dir = f"/Workspace/Users/{current_user}/{COURSE_NAME}"

print("🧹 正在执行终极环境清理...")
print("------------------------------------------")

# 1. 🗑️ 删除 Unity Catalog 表格和数据库
print(f"1. 正在删除目标数据库 {TARGET_CATALOG}.{TARGET_SCHEMA}...")
try:
    spark.sql(f"DROP SCHEMA IF EXISTS {TARGET_CATALOG}.{TARGET_SCHEMA} CASCADE")
    print("   ✅ 目标数据库已清除。")
except Exception as e:
    print(f"   ⚠️ 数据库清理失败 (可能是权限问题)。错误: {e}")

# 2. 🗑️ 彻底删除 Volume 对象 (解决了底层残留问题)
print(f"2. 正在彻底删除 Volume 对象 {CATALOG_NAME}.{SCHEMA_NAME}.{VOLUME_NAME}...")
try:
    # DROP VOLUME 彻底删除 Volume 及其所有底层数据
    spark.sql(f"DROP VOLUME IF EXISTS {CATALOG_NAME}.{SCHEMA_NAME}.{VOLUME_NAME}")
    print("   ✅ Volume 对象已彻底删除。")
except Exception as e:
    print(f"   ⚠️ Volume 删除失败 (可能是权限问题)。错误: {e}")

# 3. 🗑️ 删除用户工作区文件夹
print(f"3. 正在删除工作区文件夹 {user_workspace_dir}...")
if os.path.exists(user_workspace_dir):
    shutil.rmtree(user_workspace_dir, ignore_errors=True)
    print("   ✅ 工作区文件夹已清除。")
else:
    print("   ℹ️ 工作区文件夹不存在，跳过清理。")

print("------------------------------------------")
print("🎉 终极清理完成。Volume 已被彻底删除，准备重新创建。")

🧹 正在执行终极环境清理...
------------------------------------------
1. 正在删除目标数据库 workspace.course_db...
   ✅ 目标数据库已清除。
2. 正在彻底删除 Volume 对象 workspace.default.course_data...
   ✅ Volume 对象已彻底删除。
3. 正在删除工作区文件夹 /Workspace/Users/bdaconsulting1098@gmail.com/Data_Course_101...
   ✅ 工作区文件夹已清除。
------------------------------------------
🎉 终极清理完成。Volume 已被彻底删除，准备重新创建。


In [0]:
# ================= 配置区域 (无需修改) =================
GITHUB_REPO_URL = "https://github.com/bdaconsulting1098-creator/bda_course.git"
COURSE_NAME = "Data_Course_101"  # 用户文件夹里显示的项目名
CATALOG_NAME = "workspace"       # 目标 Catalog
SCHEMA_NAME = "default"          # 目标 Schema
VOLUME_NAME = "course_data"      # 存放数据的 Volume

# 定义数据映射: { GitHub里的路径 : Volume里的子文件夹名 }
DATA_MAPPINGS = {
    "BDA1/data": "BDA1_Data",
    "BDA2/data": "BDA2_Data",
    "database_data": "Database_Data"
}

# 定义需要复制给学生的课件文件夹 (排除 data)
COURSE_FOLDERS = ["BDA1", "BDA2"]

# ================= 自动执行逻辑 =================
import os
import shutil

# 1. 动态获取当前用户身份
current_user = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
print(f"👋 欢迎, {current_user}! 正在为您准备课程环境...")

# 2. 定义路径
temp_clone_dir = f"/tmp/{COURSE_NAME}_temp"
user_workspace_dir = f"/Workspace/Users/{current_user}/{COURSE_NAME}"
volume_path = f"/Volumes/{CATALOG_NAME}/{SCHEMA_NAME}/{VOLUME_NAME}/"

# 3. 【新增功能】尝试自动创建 Volume
print(f"🛠️  正在检查 Volume 环境...")
try:
    # 使用 Spark SQL 创建 Volume (如果不存)
    spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG_NAME}.{SCHEMA_NAME}.{VOLUME_NAME}")
    print(f"   ✅ Volume 已准备就绪: {CATALOG_NAME}.{SCHEMA_NAME}.{VOLUME_NAME}")
except Exception as e:
    print(f"   ⚠️ 自动创建 Volume 失败 (可能是权限不足)，尝试直接访问...")
    print(f"   错误信息: {str(e)}")

# 4. 清理旧环境 & 克隆仓库
print(f"⬇️  正在从 GitHub 克隆资料...")
if os.path.exists(temp_clone_dir):
    shutil.rmtree(temp_clone_dir, ignore_errors=True)

# 使用 magic command 运行 shell 命令
result = os.system(f"git clone {GITHUB_REPO_URL} {temp_clone_dir}")

if result != 0:
    raise Exception("❌ GitHub 克隆失败，请检查 URL 或网络连接。")

# 5. 部署数据到 Volume
print(f"📂 正在部署数据到 Volume: {volume_path} ...")
# 再次确保 OS 层面的目录存在
os.makedirs(volume_path, exist_ok=True)

for repo_subpath, volume_subfolder in DATA_MAPPINGS.items():
    src_path = os.path.join(temp_clone_dir, repo_subpath)
    dst_path = os.path.join(volume_path, volume_subfolder)
    
    if os.path.exists(src_path):
        # 如果目标文件夹已存在，先删除旧的
        if os.path.exists(dst_path):
            shutil.rmtree(dst_path)
        # 复制整个文件夹到 Volume
        shutil.copytree(src_path, dst_path)
        print(f"   ✅ 数据已同步: {volume_subfolder}")
    else:
        print(f"   ⚠️ 警告: 在仓库中未找到 {repo_subpath}，跳过。")

# 6. 部署 Notebooks 到用户 Workspace
print(f"📘 正在部署课件到 Workspace: {user_workspace_dir} ...")

# 过滤函数：复制时忽略名为 'data'、'__pycache__' 及隐藏文件夹
def ignore_unwanted_folders(dir, files):
    ignore_list = []
    for f in files:
        if f == 'data' or f == '__pycache__' or f.startswith('.'):
            ignore_list.append(f)
    return ignore_list

for folder in COURSE_FOLDERS:
    src_path = os.path.join(temp_clone_dir, folder)
    dst_path = os.path.join(user_workspace_dir, folder)
    
    if os.path.exists(src_path):
        if os.path.exists(dst_path):
            shutil.rmtree(dst_path)
        
        # 复制文件夹 (自动剔除 data、__pycache__、隐藏文件夹)
        shutil.copytree(src_path, dst_path, ignore=ignore_unwanted_folders)
        print(f"   ✅ 课件已安装: {folder}")
    else:
        print(f"   ⚠️ 未找到课件文件夹: {folder}")

# 7. 清理临时文件
shutil.rmtree(temp_clone_dir, ignore_errors=True)

print("="*40)
print(f"🚀 全部完成！")
print(f"1. 数据路径: {volume_path}")
print(f"2. 课件路径: {user_workspace_dir}")
print("👉 请刷新左侧 Workspace 查看您的文件。")



👋 欢迎, bdaconsulting1098@gmail.com! 正在为您准备课程环境...
🛠️  正在检查 Volume 环境...
   ✅ Volume 已准备就绪: workspace.default.course_data
⬇️  正在从 GitHub 克隆资料...


Cloning into '/tmp/Data_Course_101_temp'...


📂 正在部署数据到 Volume: /Volumes/workspace/default/course_data/ ...


Updating files: 100% (1698/1698), done.


   ✅ 数据已同步: BDA1_Data
   ✅ 数据已同步: BDA2_Data
   ✅ 数据已同步: Database_Data
📘 正在部署课件到 Workspace: /Workspace/Users/bdaconsulting1098@gmail.com/Data_Course_101 ...
   ✅ 课件已安装: BDA1
   ✅ 课件已安装: BDA2
🚀 全部完成！
1. 数据路径: /Volumes/workspace/default/course_data/
2. 课件路径: /Workspace/Users/bdaconsulting1098@gmail.com/Data_Course_101
👉 请刷新左侧 Workspace 查看您的文件。


In [0]:
# ================= 自动执行逻辑 (第二阶段：入库) =================
# 必须依赖 Cell 1 成功运行后定义的变量和路径

import os
import concurrent.futures
from threading import Lock

# 定义路径 (重定义关键变量)
CATALOG_NAME = "workspace"
SCHEMA_NAME = "default"
VOLUME_NAME = "course_data"
TARGET_CATALOG = "workspace"
TARGET_SCHEMA = "course_db"
DATABASE_DATA_VOLUME_FOLDER = "Database_Data" 

volume_base_path = f"/Volumes/{CATALOG_NAME}/{SCHEMA_NAME}/{VOLUME_NAME}"
database_volume_path = f"{volume_base_path}/{DATABASE_DATA_VOLUME_FOLDER}/"

print(f"🔄 正在将 {DATABASE_DATA_VOLUME_FOLDER} 中的文件转为 Delta 表...")

progress_lock = Lock()

def sanitize_column_names(df):
    forbidden_chars = [' ', ',', ';', '{', '}', '(', ')', '\n', '\t', '=']
    new_columns = []
    col_map = {}
    for col in df.columns:
        new_col = col
        for ch in forbidden_chars:
            new_col = new_col.replace(ch, '_')
        # Ensure uniqueness if duplicate after sanitization
        orig_new_col = new_col
        suffix = 1
        while new_col in new_columns:
            new_col = f"{orig_new_col}_{suffix}"
            suffix += 1
        new_columns.append(new_col)
        col_map[col] = new_col
    # Print warning if any columns were renamed
    renamed = [(k, v) for k, v in col_map.items() if k != v]
    if renamed:
        print(f"      ⚠️ 列名已重命名: {renamed}")
    return df.toDF(*new_columns)

def process_file(file_name):
    table_name = file_name.replace('.', '_').replace('-', '_').split('_csv')[0]
    source_file_path = os.path.join(database_volume_path, file_name)
    target_table = f"{TARGET_CATALOG}.{TARGET_SCHEMA}.{table_name}"
    with progress_lock:
        print(f"   -> 正在创建表: {table_name}")
    try:
        df = (spark.read
              .option("header", "true")
              .option("inferSchema", "true")
              .option("delimiter", ",")
              .csv(source_file_path))
        if len(df.columns) == 0:
            with progress_lock:
                print(f"      ⚠️ 文件 {file_name} 没有有效的列，已跳过 (可能是空文件或非数据文件)。")
            return
        df = sanitize_column_names(df)
        # 直接写入 Delta 表，覆盖原表
        df.write.format("delta").mode("overwrite").saveAsTable(target_table)
        with progress_lock:
            print(f"      ✅ Delta 表 {table_name} 已创建并导入成功。")
    except Exception as e:
        with progress_lock:
            print(f"      ❌ 表 {table_name} 导入失败。请检查文件格式是否为 CSV/Parquet/JSON。")
            print(f"         错误: {str(e)}")

# 确保目标 Schema 存在
try:
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {TARGET_CATALOG}.{TARGET_SCHEMA}")
    print(f"✅ 目标 Schema 已准备: {TARGET_CATALOG}.{TARGET_SCHEMA}")
except Exception as e:
    print(f"⚠️ Schema 创建失败: {e}")

# 确保目标文件夹存在
if not os.path.exists(database_volume_path):
    print("❌ 转换失败：database_data 文件夹在 Volume 中不存在。请检查 Cell 1 是否成功运行。")
else:
    try:
        file_names = os.listdir(database_volume_path)
    except Exception as e:
        print(f"❌ 访问 Volume 路径失败，请检查集群是否挂载正常。错误: {e}")
        file_names = []
    data_files = [f for f in file_names if not f.startswith('.') and os.path.isfile(os.path.join(database_volume_path, f))]
    if not data_files:
        print("❌ 转换失败：os.listdir 仍然未找到任何文件。请再次手动刷新 Catalog Explorer，并确认文件是否为空。")
    else:
        # 并行处理文件
        with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
            executor.map(process_file, data_files)

print("="*40)
print(f"🎉 第二阶段完成！数据入库成功。")
print(f"请刷新 Catalog Explorer，并在 {TARGET_CATALOG}.{TARGET_SCHEMA} 下查看您的新表格。")

🔄 正在将 Database_Data 中的文件转为 Delta 表...
✅ 目标 Schema 已准备: workspace.course_db
   -> 正在创建表: HumanResources_Department
   -> 正在创建表: HumanResources_Employee
   -> 正在创建表: HumanResources_EmployeeDepartmentHistory
   -> 正在创建表: HumanResources_EmployeePayHistory
   -> 正在创建表: HumanResources_JobCandidate
   -> 正在创建表: HumanResources_Shift
   -> 正在创建表: HumanResources_vEmployee
   -> 正在创建表: HumanResources_vEmployeeDepartment
      ✅ Delta 表 HumanResources_Shift 已创建并导入成功。
   -> 正在创建表: HumanResources_vEmployeeDepartmentHistory
      ✅ Delta 表 HumanResources_Employee 已创建并导入成功。
   -> 正在创建表: HumanResources_vJobCandidate
      ✅ Delta 表 HumanResources_EmployeePayHistory 已创建并导入成功。
   -> 正在创建表: HumanResources_vJobCandidateEducation
      ✅ Delta 表 HumanResources_Department 已创建并导入成功。
   -> 正在创建表: HumanResources_vJobCandidateEmployment
      ✅ Delta 表 HumanResources_EmployeeDepartmentHistory 已创建并导入成功。
   -> 正在创建表: Person_Address
      ✅ Delta 表 HumanResources_JobCandidate 已创建并导入成功。
   -> 正在创建表: Person_AddressTy